In [1]:
import pandas as pd
import numpy as np
import re
import csv
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import pickle
import time
import random

In [2]:
df = pd.read_excel('Unbenannte Tabelle.xlsx')
df = df[['linkedInProfileUrl']].drop_duplicates().reset_index(drop=True)
df

,linkedInProfileUrl
0,https://www.linkedin.com/in/ACwAAENRRwgBL-TkIh...
1,https://www.linkedin.com/in/ACwAABiqAQMBguBhpm...
2,https://www.linkedin.com/in/ACwAACGYi7IB0trJpY...
3,https://www.linkedin.com/in/ACwAABiFGAEB4_PZtd...
4,https://www.linkedin.com/in/ACwAACRqeTYB-YU8Af...
...,...
6719,https://www.linkedin.com/in/ACwAADUCMUEBpUgliz...
6720,https://www.linkedin.com/in/ACwAAC1x5GgBl91AhW...
6721,https://www.linkedin.com/in/ACwAABLzYekBjKhVr4...
6722,https://www.linkedin.com/in/ACwAABHiTFcB776Yr5...


In [81]:
# launch browser and save authentication
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--start-maximized')
# chrome_options.add_argument('headless')
# browser = webdriver.Chrome(options=chrome_options)
browser = webdriver.Chrome()
# browser.maximize_window()
browser.get('https://www.linkedin.com/feed/?trk=homepage-basic_sign-in-submit')

In [39]:
# cookies = browser.get_cookies()
# pickle.dump(cookies,open('cookies1.pkl','wb'))

In [82]:
cookies = pickle.load(open('cookies1.pkl','rb'))
for cookie in cookies:
    browser.add_cookie(cookie)

browser.get('https://www.linkedin.com/feed/?trk=homepage-basic_sign-in-submit')

In [61]:
def write_append_data(data):

    if os.path.exists(path='linkedIn.csv'):

        with open('linkedIn.csv', mode='a',newline='', encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerows(data)
    else:
        with open('linkedIn.csv',newline='', mode='w', encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(['orginal_page','image_link','about_link','name','title','location','contact_info','followers_link','connections_link','followers','connections','company_name','company_location',
                             'years_in_company','num_positions','company_link','education','education_degree','education_date_range','num_education','education_link','about','skills','num_skills','skills_link',
                             'num_certifications','certifications','certifications_link','num_languages','interest_companies'])
            writer.writerows(data)

In [62]:
def delays(duration):
    """
    Generates a random delay duration based on the specified category.

    Parameters:
    - duration (str): The category of delay ('s' for small, 'l' for load page, 'm' for mid, 'b' for big).

    Returns:
    float: The randomly generated delay duration in seconds.

    Example:
    >>> delays('s')
    2.345

    Note:
    - 's' corresponds to a delay between 1 and 5 seconds.
    - 'l' corresponds to a delay between 30 and 60 seconds.
    - 'm' corresponds to a delay between 1 minute and 5 minutes.
    - 'b' corresponds to a delay between 5 minutes and 10 minutes.
    """
    # 1 sec to 3 sec
    small_delay = random.uniform(1, 3)
    
    # 30 sec to 60 sec
    load_page_delay = random.randint(15, 60)
    
    # 1 min to 5 min
    mid_delay = random.randint(60, 300)
    
    # 5 min to 10 min
    big_delay = random.randint(300, 600)

    if duration == 's':
        return small_delay
    elif duration == 'l':
        return load_page_delay
    elif duration == 'm':
        return mid_delay
    elif duration == 'b':
        return big_delay

def goto_the_page(url):
    """
    Opens a new browser tab, switches to it, and navigates to the specified URL.

    Parameters:
    - url (str): The URL to navigate to.

    Returns:
    None

    Example:
    >>> goto_the_page("https://www.example.com")
    """
    # Execute JavaScript to open a new browser tab
    browser.execute_script("window.open('', '_blank');")

    # Wait for a brief moment to ensure the new tab is created
    time.sleep(delays('s'))

    # Switch to the newly opened tab
    browser.switch_to.window(browser.window_handles[-1])

    # Wait for a moment to ensure the switch is complete
    time.sleep(delays('s'))

    # Navigate to the specified URL
    browser.get(url=url)

def close_switch_to_original(to_page):
    """
    Closes the current browser tab and switches back to the original tab.

    This function assumes that the browser was opened with multiple tabs.

    Returns:
    None

    Raises:
    AssertionError: If the number of browser tabs is equal to 1 after switching.
    
    Example:
    >>> close_switch_to_original()
    """

    # Check if there are still multiple tabs open
    if len(browser.window_handles) > 1:

        # Close the current browser tab
        browser.close()

        # Wait briefly to ensure the tab is closed before switching
        time.sleep(delays('s'))
        
        # Switch to the current opened tab
        browser.switch_to.window(browser.window_handles[to_page])
        
        # Wait briefly to ensure the switch is complete
        time.sleep(delays('s'))

    # # Assert that there is only one tab open after the switch
    # assert len(browser.window_handles) == 1


options = ['search','home','mynetwork','jobs','messaging','notifications','me','for business']
def press_on(OneOfFour='options'):
    """
    Performs various actions based on the specified option.

    Parameters:
    - OneOfFour (str): The option to perform an action on.

    Returns:
    None

    Example:
    >>> press_on('search')
    # Performs search-related actions.

    >>> press_on('messaging')
    # Clicks on the messaging option in the navigation bar and adds a delay.
    """

    try:
        if OneOfFour == 'options':
            print(f"Choose one of the following to click it: {options}")
            
        elif OneOfFour in ['mynetwork','jobs','messaging','notifications']:
            [i.click() for i in browser.find_elements(By.CSS_SELECTOR, 'li a[class*="app-aware-link"]') if OneOfFour in i.get_attribute('href')]
            time.sleep(delays('l'))

        elif OneOfFour == 'search':
            browser.find_element(By.CSS_SELECTOR,'div[id="global-nav-search"]').click()
            time.sleep(delays('s'))

            ActionChains(browser).send_keys(Keys.ARROW_DOWN + Keys.ENTER).perform()
            time.sleep(delays('l'))
        elif OneOfFour == 'home':
            browser.find_element(By.CSS_SELECTOR,'header[id="global-nav"] a[class="app-aware-link "]').click()
            time.sleep(delays('l'))

        elif OneOfFour == 'me':
            browser.find_elements(By.CSS_SELECTOR,'button[class*="global-nav__primary-link"]')[0].click()
            time.sleep(delays('s'))
            ActionChains(browser).move_by_offset(-5,0).click().perform()
            
        elif OneOfFour == 'for business':
            browser.find_elements(By.CSS_SELECTOR,'button[class*="global-nav__primary-link"]')[1].click()
            time.sleep(delays('s'))
            ActionChains(browser).move_by_offset(-5,0).click().perform()

        else:
            print(f"Please Enter one of the following {options}")
    except:
        pass


def if_exists(selector):
    try:
        browser.find_elements(By.CSS_SELECTOR,selector)
        browser.find_element(By.CSS_SELECTOR,selector)
        return True
    except:
        return False
    

choices = ['about','experience','licenses_and_certifications','skills','interests','education']
def click_on_elements(choice):
    if choice == 'choices':
        print(choices)
    """
    Double-clicks on a specified element if it exists.

    Parameters:
    - choice (str): The choice corresponding to the element to be clicked.

    Returns:
    None

    Example:
    >>> click_on_elements('experience')
    # Performs a double-click action on the 'experience' element.
    """
    selector = f'#{choice} ~ div span[aria-hidden="true"]'
    if if_exists(selector):
        ActionChains(browser).move_to_element(browser.find_element(By.CSS_SELECTOR,selector)).perform()
        time.sleep(delays('s'))
        ActionChains(browser).double_click(browser.find_element(By.CSS_SELECTOR,selector)).perform()
    else:
        pass

def scroll_scenarios(scenario):
    """
    Performs various scrolling scenarios based on the specified scenario.

    Parameters:
    - scenario (str): The scenario for scrolling ('normal', 'more normal', 'all', 'end', 'home').

    Returns:
    None

    Example:
    >>> scroll_scenarios('normal')
    # Performs a normal human-like scrolling scenario.

    >>> scroll_scenarios('all')
    # Scrolls through all interests on the webpage.
    """
    action = ActionChains(browser)

    def normal_human_scenario():
        numloop = np.floor(browser.execute_script("return document.body.scrollHeight") / 75 / 6)
        for _ in range(int(numloop)):
            for _ in range(6):
                browser.execute_script("window.scrollBy(0, 75);")
                action.move_by_offset(random.uniform(-1, 1), 0).perform()
                time.sleep(random.uniform(0.1, 0.25))
            time.sleep(delays('s'))

    def more_than_normal():
        numloop = np.floor(browser.execute_script("return document.body.scrollHeight") / 200 / 6)
        for _ in range(int(numloop)):
            for _ in range(6):
                browser.execute_script("window.scrollBy(0, 200);")
                action.move_by_offset(random.uniform(-1, 1), 0).perform()
                time.sleep(random.uniform(0.1, 0.25))
            time.sleep(delays('s'))

    def scroll_all_interests():
        last_height = browser.execute_script("return document.body.scrollHeight")
        numloop = np.floor(last_height / 75 / 6)
        while True:
            for _ in range(int(numloop)):
                for _ in range(6):
                    browser.execute_script("window.scrollBy(0, 75);")
                    action.move_by_offset(random.uniform(-1, 1), 0).perform()
                    time.sleep(random.uniform(0.1, 0.25))
                time.sleep(delays('s'))

            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            numloop = np.floor((new_height - last_height) / 75 / 6)
            last_height = new_height

    def go_end():
        ActionChains(browser).send_keys(Keys.END).perform()

    def go_up():
        ActionChains(browser).send_keys(Keys.HOME).perform()

    if scenario == 'normal':
        return normal_human_scenario()
    elif scenario == 'more normal':
        return more_than_normal()
    elif scenario == 'all':
        return scroll_all_interests()
    elif scenario == 'end':
        return go_end()
    elif scenario == 'home':
        return go_up()
    else:
        print(f"Invalid scenario: {scenario}")
        
def personal_page_scenarios(mcq):
    if mcq == 'about':
        browser.find_elements(By.CSS_SELECTOR,'div[class="mt2 relative"] a')[0].click() if if_exists('div[class="mt2 relative"] a') else None
        time.sleep(delays('s'))
        browser.back()
    elif mcq == 'contact info':
        browser.find_element(By.CSS_SELECTOR,'span[class="pv-text-details__separator t-black--light"] > a').click() if if_exists('span[class="pv-text-details__separator t-black--light"] > a') else None
        time.sleep(delays('s'))
        browser.back()
    elif mcq == 'posts':
        browser.find_element(By.CSS_SELECTOR,'#content_collections ~ footer a').click() if if_exists('#content_collections ~ footer a') else None
        time.sleep(delays('s'))
        scroll_scenarios(random.choices(['normal','more normal','end'])[0])
        browser.back()
    else:
        print('Please Enter One of the following ["about","contact info","posts"]')

def go_to_youtub():
    browser.get(url='https://www.youtube.com/')
    time.sleep(1800)
    goto_the_page('https://www.linkedin.com/feed/')
    browser.switch_to.window(browser.window_handles[0])
    browser.close()
    browser.switch_to.window(browser.window_handles[0])

In [63]:
def extract_location_company(selector):
        
        if if_exists(selector):
            selector = browser.find_elements(By.CSS_SELECTOR,selector)[0].text
            return selector
        else:
            return 'not specified'

def Location(selector):
        
        if if_exists(selector):
            selector = browser.find_elements(By.CSS_SELECTOR,selector)[0].text 
            if re.findall(r'[0-9]',selector):
                return "not specified"
            else:
                return selector
        else:
            return "not specified"
        
def years(selector):
    
    if if_exists(selector):
        selector = browser.find_elements(By.CSS_SELECTOR,selector)[0].text 
        if re.findall(r'[0-9]',selector):
            return selector
        else:
            return "not specified"
    else:
        return "not specified"

def extract_years_company(selector):
        
        if if_exists(selector):
            selector = browser.find_elements(By.CSS_SELECTOR,selector)[0].text 
            return selector
        else:
            return 'not specified'
        
def Extract_num_skills(url,numskills):
        
        goto_the_page(url=url)
        # time.sleep(delays('l'))
        element_to_scroll_to = (By.CSS_SELECTOR, 'div[class="scaffold-finite-scroll__content"] > ul[class*="pvs-list"] > li  div[data-view-name="profile-component-entity"] div[class="display-flex flex-row justify-space-between"]  span[aria-hidden="true"]')
        WebDriverWait(browser,random.uniform(10, 20)).until(EC.presence_of_element_located(element_to_scroll_to))
        
        scroll_scenarios('all')

        selector = 'div[class="scaffold-finite-scroll__content"] > ul[class*="pvs-list"] > li  div[data-view-name="profile-component-entity"] div[class="display-flex flex-row justify-space-between"]  span[aria-hidden="true"]'
        skills = browser.find_elements(By.CSS_SELECTOR,selector)[:numskills]

        all_skills = ''
        for ski in skills:
            all_skills += f"{ski.text} | "
        
        time.sleep(delays('s'))
        close_switch_to_original(1)

        return  all_skills
    
def Extract_num_certifications(url):
        
        goto_the_page(url=url)
        # time.sleep(delays('l'))
        element_to_scroll_to = (By.CSS_SELECTOR,'div[class="scaffold-finite-scroll__content"] > ul[class*="pvs-list"] > li  div[data-view-name="profile-component-entity"] div[class="display-flex flex-row justify-space-between"] ')
        WebDriverWait(browser,random.uniform(10, 20)).until(EC.presence_of_element_located(element_to_scroll_to))
        
        scroll_scenarios('all')
        
        selector = 'div[class="scaffold-finite-scroll__content"] > ul[class*="pvs-list"] > li  div[data-view-name="profile-component-entity"] div[class="display-flex flex-row justify-space-between"] '
        cert = [select.find_elements(By.CSS_SELECTOR,'span[aria-hidden="true"]')[0].text for select in browser.find_elements(By.CSS_SELECTOR,selector)]
        all_cert = ''
        for car in cert:
                all_cert += f"{car} | "

        time.sleep(delays('s'))
        close_switch_to_original(1)
        return  all_cert
    
def Extract_interests(url):
        
        goto_the_page(url=url)
        # time.sleep(delays('l'))
        element_to_scroll_to = (By.CSS_SELECTOR,'div[class="display-flex flex-wrap align-items-center full-height"] span[aria-hidden="true"]')
        WebDriverWait(browser,random.uniform(10, 20)).until(EC.presence_of_element_located(element_to_scroll_to))

        scroll_scenarios('all')

        interests = [i.text for i in browser.find_elements(By.CSS_SELECTOR,'div[class="display-flex flex-wrap align-items-center full-height"] span[aria-hidden="true"]')]
        all_interests = ''
        for inter in interests:
            if inter != '':
                all_interests += f"{inter} | "
            else:
                pass

        time.sleep(delays('s'))
        close_switch_to_original(1)
        return all_interests

In [ ]:
# 0 to 1000
# [600,650,700,750,800,850,900,950]

In [ ]:
urls = df[df.index >= 547]['linkedInProfileUrl'].tolist()
rows = []
past_page = ''
for i ,url in enumerate(urls,547):
        if i == 1000:
            break
        goto_the_page(url=url)
        WebDriverWait(browser,random.uniform(10, 20)).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[class="pv-top-card__non-self-photo-wrapper ml0"] > button > img')))
        # time.sleep(delays('l'))

        personal_page_scenarios(random.choices(["about","contact info","posts"])[0])
        time.sleep(delays('s'))

        scroll_scenarios(random.choices(['normal','more normal','all','end'])[0])

        image_link = browser.find_element(By.CSS_SELECTOR,'div[class="pv-top-card__non-self-photo-wrapper ml0"] > button > img').get_attribute('src')
        about_link = browser.find_elements(By.CSS_SELECTOR,'div[class="mt2 relative"] a')[0].get_attribute('href')
        name = browser.find_element(By.CSS_SELECTOR,'div[class*="ph5 "] div[class="mt2 relative"] a > h1').text
        title = browser.find_element(By.CSS_SELECTOR,'div[class="text-body-medium break-words"]').text
        location = browser.find_element(By.CSS_SELECTOR,'span[class="text-body-small inline t-black--light break-words"]').text

        click_on_elements(random.choices(choices)[0])
        ActionChains(browser).move_by_offset(0,0).click().perform()

        time.sleep(delays('s'))
        contact_info = browser.find_element(By.CSS_SELECTOR,'span[class="pv-text-details__separator t-black--light"] > a').get_attribute('href') if if_exists('span[class="pv-text-details__separator t-black--light"] > a') else 'not specified'
        followers_link = "https://www.linkedin.com"+ browser.find_element(By.CSS_SELECTOR,'li[class="text-body-small t-black--light inline-block"] a').get_attribute('href') if if_exists('li[class="text-body-small t-black--light inline-block"] a') else 'not specified'
        connections_link = "https://www.linkedin.com"+ browser.find_element(By.CSS_SELECTOR,'li[class="text-body-small"] > a').get_attribute('href') if if_exists('li[class="text-body-small"] > a') else 'not specified'
        connections = browser.find_element(By.CSS_SELECTOR,'li[class="text-body-small"] span.t-bold').text if if_exists('li[class="text-body-small"] span.t-bold') else 0
        followers = int(re.sub(r'[^0-9]','',browser.find_element(By.CSS_SELECTOR,'#content_collections ~ div p > span[aria-hidden="true"]').text)) if if_exists('#content_collections ~ div p > span[aria-hidden="true"]') else browser.find_element(By.CSS_SELECTOR,'li[class="text-body-small"] span.t-bold').text if connections != 0 else 0

        click_on_elements(random.choices(choices)[0])
        ActionChains(browser).move_by_offset(0,0).click().perform()

        time.sleep(delays('s'))
        company_name = browser.find_elements(By.CSS_SELECTOR,'#experience ~ div div[class="display-flex "] span')[0].text if if_exists('#experience ~ div div[class="display-flex "] span') else 'not specified'
        if if_exists('#experience ~ div div[class="display-flex flex-row justify-space-between"]'):
            company_location =  extract_location_company('#experience ~ div div[class="display-flex flex-row justify-space-between"] span[class="t-14 t-normal"] > span') if len(browser.find_elements(By.CSS_SELECTOR,'#experience ~ div div[class="display-flex flex-row justify-space-between"]')[0].find_elements(By.CSS_SELECTOR,'span[class*="t-14 t-normal"]')) >= 2 else Location('#experience ~ div div[class="display-flex flex-row justify-space-between"] span[class="t-14 t-normal"] > span')
            years_in_company = extract_years_company('#experience ~ div div[class="display-flex flex-row justify-space-between"] span[class="t-14 t-normal t-black--light"] > span') if len(browser.find_elements(By.CSS_SELECTOR,'#experience ~ div div[class="display-flex flex-row justify-space-between"]')[0].find_elements(By.CSS_SELECTOR,'span[class*="t-14 t-normal"]')) >= 2  else years('#experience ~ div div[class="display-flex flex-row justify-space-between"] span[class="t-14 t-normal"] > span')
        else:
            print(i , url)
            company_location = 'not specified'
            years_in_company = 'not specified'
        num_positions = int(re.sub(r'[^1-9]','',browser.find_element(By.CSS_SELECTOR,'#experience ~ div span[class="pvs-navigation__text"]').text)) if if_exists('#experience ~ div span[class="pvs-navigation__text"]') else len(browser.find_elements(By.CSS_SELECTOR,'#experience ~ div ul[class*="pvs-list"] > li > div[data-view-name="profile-component-entity"]')) if if_exists('#experience ~ div ul[class*="pvs-list"] > li > div[data-view-name="profile-component-entity"]') else 0
        company_link = browser.find_elements(By.CSS_SELECTOR,'#experience ~ div div[data-view-name="profile-component-entity"] a')[0].get_attribute('href') if if_exists('#experience ~ div div[data-view-name="profile-component-entity"] a') else 'not specified'

        click_on_elements(random.choices(choices)[0])
        ActionChains(browser).move_by_offset(0,0).click().perform()
        
        time.sleep(delays('s'))
        education = browser.find_elements(By.CSS_SELECTOR,'#education ~ div div[class="display-flex flex-row justify-space-between"] >a span')[0].text if if_exists('#education ~ div div[class="display-flex flex-row justify-space-between"] >a span') else 'not specified'
        education_degree = browser.find_elements(By.CSS_SELECTOR,'div[id="education"] ~ div div[class="display-flex flex-row justify-space-between"] > a span span[aria-hidden="true"]')[0].text if if_exists('div[id="education"] ~ div div[class="display-flex flex-row justify-space-between"] > a span span[aria-hidden="true"]') else 'not specified'
        education_date_range = browser.find_elements(By.CSS_SELECTOR,'div[id="education"] ~ div div[class="display-flex flex-row justify-space-between"] > a span[class="t-14 t-normal t-black--light"] > span')[0].text if if_exists('div[id="education"] ~ div div[class="display-flex flex-row justify-space-between"] > a span[class="t-14 t-normal t-black--light"] > span') else 'not specified'
        num_education = int(re.sub(r'[^1-9]','', browser.find_element(By.CSS_SELECTOR,'a[id="navigation-index-see-all-education"] > span').text)) if if_exists('a[id="navigation-index-see-all-education"]') else len(browser.find_elements(By.CSS_SELECTOR,'#education~ div ul[class*="pvs-list"] > li')) if if_exists('#education~ div ul[class*="pvs-list"] > li') else 0
        education_link = browser.find_elements(By.CSS_SELECTOR,'div[id="education"] ~ div div[class="display-flex flex-row justify-space-between"] > a')[0].get_attribute('href') if if_exists('div[id="education"] ~ div div[class="display-flex flex-row justify-space-between"] > a') else 'not specified'

        click_on_elements(random.choices(choices)[0])
        ActionChains(browser).move_by_offset(0,0).click().perform()

        time.sleep(delays('s'))
        about = browser.find_elements(By.CSS_SELECTOR,'#about ~ div[class*="display-flex"] span')[0].text if if_exists('#about ~ div[class*="display-flex"] span') else 'not specified'
        num_skills = int(re.sub(r'[^1-9]','',browser.find_element(By.CSS_SELECTOR,'#skills ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a > span').text)) if if_exists('#skills ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a > span') else len(browser.find_elements(By.CSS_SELECTOR,'#skills ~ div ul[class*="pvs-list"] > li > div[data-view-name="profile-component-entity"]')) if if_exists('#skills ~ div ul[class*="pvs-list"] > li > div[data-view-name="profile-component-entity"]') else 0
        skills = Extract_num_skills(browser.find_element(By.CSS_SELECTOR,'#skills ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a').get_attribute('href'),num_skills) if if_exists('#skills ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a') else ' | '.join([i for i in map(lambda x: x.text,browser.find_elements(By.CSS_SELECTOR,'#skills ~ div ul[class*="pvs-list"] > li > div[data-view-name="profile-component-entity"] a span[aria-hidden="true"]')) if 'endorsements' not in i]) if if_exists('#skills ~ div ul[class*="pvs-list"] > li > div[data-view-name="profile-component-entity"] a span[aria-hidden="true"]') else 'not specified'
        skills_link = browser.find_element(By.CSS_SELECTOR,'#skills ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a').get_attribute('href') if if_exists('#skills ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a') else 'not specified'

        click_on_elements(random.choices(choices)[0])
        ActionChains(browser).move_by_offset(0,0).click().perform()
        
        time.sleep(delays('s'))
        num_certifications = int(re.sub(r'[^1-9]','',browser.find_element(By.CSS_SELECTOR,'#licenses_and_certifications ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a > span').text)) if if_exists('#licenses_and_certifications ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a > span') else len(browser.find_elements(By.CSS_SELECTOR,'#licenses_and_certifications ~ div ul[class*="pvs-list"] > li > div[data-view-name="profile-component-entity"]')) if if_exists('#licenses_and_certifications ~ div ul[class*="pvs-list"] > li > div[data-view-name="profile-component-entity"]') else 0
        certifications = Extract_num_certifications(browser.find_element(By.CSS_SELECTOR,'#licenses_and_certifications ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a').get_attribute('href')) if if_exists('#licenses_and_certifications ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a') else ' | '.join([i for i in map(lambda x: x.find_element(By.CSS_SELECTOR,'span[aria-hidden="true"]').text,browser.find_elements(By.CSS_SELECTOR,'#licenses_and_certifications ~ div ul[class*="pvs-list"] > li > div[data-view-name="profile-component-entity"] div[class="display-flex flex-row justify-space-between"]'))]) if if_exists('#licenses_and_certifications ~ div ul[class*="pvs-list"] > li > div[data-view-name="profile-component-entity"] div[class="display-flex flex-row justify-space-between"] ') else 'not specified'
        certifications_link = browser.find_element(By.CSS_SELECTOR,'#licenses_and_certifications ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a').get_attribute('href') if if_exists('#licenses_and_certifications ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a') else 'not specified'

        num_languages = int(re.sub(r'[^1-9]','',browser.find_element(By.CSS_SELECTOR,'#languages ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a > span').text)) if if_exists('#languages ~ div[class="pvs-list__outer-container"] div[class*="pvs-list__footer-wrapper"] a > span') else len(browser.find_elements(By.CSS_SELECTOR,'#languages ~ div ul[class*="pvs-list"] > li')) if if_exists('#languages ~ div ul[class*="pvs-list"] > li') else 0
        time.sleep(delays('s'))

        click_on_elements(random.choices(choices)[0])
        ActionChains(browser).move_by_offset(0,0).click().perform()

        if if_exists('#interests'):

            if 'Companies' in [i.text.strip() for i in browser.find_elements(By.CSS_SELECTOR,'#interests ~ div div[role="tablist"] > button > span[aria-hidden="true"]')]:
                company = [i for i in browser.find_elements(By.CSS_SELECTOR,'#interests ~ div div[role="tablist"] > button > span[aria-hidden="true"]') if 'Companies' == i.text.strip()][0]
                try:
                    company.click()
                except Exception as e:
                    print(e , i , url)
                time.sleep(delays('s'))
                interest_companies = Extract_interests(browser.find_element(By.CSS_SELECTOR,'#interests ~ div div[class="artdeco-tabpanel active ember-view"] div[class*="pvs-list__footer-wrapper"] a').get_attribute('href')) if if_exists('#interests ~ div div[class="artdeco-tabpanel active ember-view"] div[class*="pvs-list__footer-wrapper"] a')  else ' | '.join([i.text for i in browser.find_elements(By.CSS_SELECTOR,'#interests ~ div div[class="artdeco-tabpanel active ember-view"] li div[class="display-flex "] span[aria-hidden="true"]')]) 
            else:
                interest_companies = 'not specified'
        else:
            interest_companies = 'not specified'
        rows.append([url,image_link,about_link,name,title,location,contact_info,followers_link,connections_link,followers,connections,company_name,company_location,
                                                years_in_company,num_positions,company_link,education,education_degree,education_date_range,num_education,education_link,about,skills,num_skills,skills_link,
                                                num_certifications,certifications,certifications_link,num_languages,interest_companies])

        close_switch_to_original(0)
        
        pages = len(browser.window_handles)
        for _ in range(pages):
            if pages != 1:
                browser.switch_to.window(browser.window_handles[-1])
                time.sleep(delays('s'))
                browser.close()
                browser.switch_to.window(browser.window_handles[0])
                time.sleep(delays('s'))
            pages = len(browser.window_handles)

        past_page = browser.current_url
        press = random.choices(options)[0]
        press_on(press)
        current_page = browser.current_url
        
        if past_page == current_page:
            x_options = [i for i in options if i != press]
            press_on(random.choices(x_options)[0])

        if i in [600,650,700,750,800,850,900,950]:
            go_to_youtub()

        print('===' * 50)
        print(F"ROW => {i}")
        print([url,image_link,about_link,name,title,location,contact_info,followers_link,connections_link,followers,connections,company_name,company_location,
                    years_in_company,num_positions,company_link,education,education_degree,education_date_range,num_education,education_link,about,
                    skills,num_skills,skills_link,num_certifications,certifications,certifications_link,num_languages,interest_companies])
        print('===' * 50)

In [84]:
len(rows)

110

In [85]:
write_append_data(rows)
# 436

In [3]:
d = pd.read_csv('linkedIn.csv')
d

,orginal_page,image_link,about_link,name,title,location,contact_info,followers_link,connections_link,followers,...,education_link,about,skills,num_skills,skills_link,num_certifications,certifications,certifications_link,num_languages,interest_companies
0,https://www.linkedin.com/in/ACwAAENRRwgBL-TkIh...,https://media.licdn.com/dms/image/D4E03AQHP-9_...,https://www.linkedin.com/in/sebastian-heger-00...,Sebastian Heger,Leiter Wertpapier / Handel @ SPK Neumarkt-Pars...,"Kemnath, Bavaria, Germany",https://www.linkedin.com/in/sebastian-heger-00...,not specified,not specified,351,...,https://www.linkedin.com/company/15091809/,not specified,Portfoliomanagement | Ganzheitliche Finanzplan...,19,https://www.linkedin.com/in/sebastian-heger-00...,1,TOEFL iBT | Score 93,not specified,1,Bayerische Landesbank | LBBW | Helaba | DekaBa...
1,https://www.linkedin.com/in/ACwAABiqAQMBguBhpm...,https://media.licdn.com/dms/image/C4E03AQGzIHb...,https://www.linkedin.com/in/janheckmann/overla...,Jan Heckmann,Relationship Manager im Private Banking bei Di...,Greater Bremen Area,https://www.linkedin.com/in/janheckmann/overla...,not specified,not specified,349,...,https://www.linkedin.com/company/730179/,not specified,Wertpapiere | Finanzdienstleistungen | Bankwes...,16,https://www.linkedin.com/in/janheckmann/detail...,0,not specified,not specified,2,Harvard Business Review | MarketWatch | Fortun...
2,https://www.linkedin.com/in/ACwAACGYi7IB0trJpY...,https://media.licdn.com/dms/image/C4D03AQGXpUW...,https://www.linkedin.com/in/dr-christian-tonne...,Dr. Christian Tonnesen,Direktor Wertpapier- und Transactions-Services...,Greater Hamburg Area,https://www.linkedin.com/in/dr-christian-tonne...,not specified,not specified,432,...,https://www.linkedin.com/search/results/all/?k...,not specified,not specified,0,not specified,0,not specified,not specified,2,Victoria University of Wellington | Deutsche W...
3,https://www.linkedin.com/in/ACwAABiFGAEB4_PZtd...,https://media.licdn.com/dms/image/C4E03AQH3rHW...,https://www.linkedin.com/in/markus-p-715928b5/...,Markus P.,Leiter Produkte und Prozesse Wertpapiere / Vor...,"Idstein, Hesse, Germany",https://www.linkedin.com/in/markus-p-715928b5/...,not specified,not specified,609,...,https://www.linkedin.com/search/results/all/?k...,not specified,Strategisches Management | Projektmanagement |...,12,https://www.linkedin.com/in/markus-p-715928b5/...,3,Zertifizierter Vertriebsmanager / Sales Manage...,https://www.linkedin.com/in/markus-p-715928b5/...,2,Capital Group | Deutsche Bundesbank | DekaBank...
4,https://www.linkedin.com/in/ACwAACRqeTYB-YU8Af...,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP//...",https://www.linkedin.com/in/michael-naglak-532...,Michael Naglak,Bereichsleiter Treasury / Wertpapiere bei Spar...,"Witten, North Rhine-Westphalia, Germany",https://www.linkedin.com/in/michael-naglak-532...,not specified,not specified,251,...,https://www.linkedin.com/company/74085/,not specified,Depot A Management | Portfoliomanagement | Tre...,14,https://www.linkedin.com/in/michael-naglak-532...,0,not specified,not specified,0,Allianz Global Investors | LBBW | Deutsche Bun...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,https://www.linkedin.com/in/ACwAADuAoZcBPmuU8d...,https://media.licdn.com/dms/image/D4E03AQHBsjO...,https://www.linkedin.com/in/jakob-henninger-78...,Jakob Henninger,Privatkundenberater Volksbank Lahr eG,Germany,https://www.linkedin.com/in/jakob-henninger-78...,not specified,not specified,129,...,https://www.linkedin.com/company/7483538/,not specified,not specified,0,not specified,0,not specified,not specified,0,Frankfurter Allgemeine Zeitung | Deutsche Börs...
1512,https://www.linkedin.com/in/ACwAAAg8J28BSeaCx2...,https://media.licdn.com/dms/image/C4D03AQHw8oM...,https://www.linkedin.com/in/stephan-kunkel-83b...,Stephan Kunkel,Zukunft | Gemeinsam | Gestalten,"Niedereschach, Baden-Württemberg, Germany",https://www.linkedin.com/in/stephan-kunkel-83b...,not specifi